### I94 Immigration Analytics

Importing pyspark functions and types

In [1]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1600656798072_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Installing Pandas in EMR

In [6]:
sc.install_pypi_package('pandas')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Reading the I94 facts table

In [2]:
df_i94 = spark.read.parquet("s3://us-immigration-dl/I94_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_i94.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(cicid=3558942.0, i94cit=249.0, i94res=249.0, iso_arrdate='2016-08-16', iso_depdate=None, iso_duedate='-D/-S', i94_visa='Student', i94_mode='Air', admnum=9438617085.0, insnum=None, i94addr_US_state=None, airline='DL', fltno='63', visatype='F1', i94bir=25.0, gender='M', i94yr=2016.0, i94mon=8.0, i94port='OMA')]

#### Reading the POE table

In [4]:
df_poe = spark.read.options(delimiter=",", header=True)\
                .csv("s3://us-immigration-cleaned-data/I94_ports.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_poe.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(code='CLG', port=None, locality='Calgary', province='Alberta', territory='Canada')]

#### Reading the cities demographics table

In [6]:
df_cities_demo = spark.read.parquet("s3://us-immigration-dl/cities_demo/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_cities_demo =  df_cities_demo.withColumn("Race_rank_by_city", \
                    F.dense_rank().over(Window.partitionBy("Race").orderBy(F.desc("Count"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_cities_demo.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(Median_Age=36.0, Male_Population=4081698, Female_Population=4468707, Total_Population=8550405, Number_of_Veteranas=156961, Foreign-born=3212500, Avg_Household_Size=2.68, Race='American Indian and Alaska Native', Count=90923, Race_percent_by_city=1.0633765301175793, Race_rank_by_city=1, code='NYC')]

#### Reading the Countries table

In [9]:
df_countries = spark.read.options(delimiter=",", header=True, inferSchema=True)\
                .csv("s3://us-immigration-cleaned-data/Country_Codes.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_countries.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(cit_res_code=582, country='mexico', continent='North America')]

#### Merging tables one by one

In [11]:
df_merged1 = df_i94.join(F.broadcast(df_countries), \
                             df_i94.i94cit==df_countries.cit_res_code, 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_merged1.limit(1).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(cicid=3558942.0, i94cit=249.0, i94res=249.0, iso_arrdate='2016-08-16', iso_depdate=None, iso_duedate='-D/-S', i94_visa='Student', i94_mode='Air', admnum=9438617085.0, insnum=None, i94addr_US_state=None, airline='DL', fltno='63', visatype='F1', i94bir=25.0, gender='M', i94yr=2016.0, i94mon=8.0, i94port='OMA', cit_res_code=249, country='iran', continent='Asia')]

### Analyzing Asian's Preferences

In [22]:
df_asia = df_merged1.filter(F.col("continent")=='Asia')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Preferences of Students for the month of august when fall academic year starts

In [25]:
df_res = df_asia.filter(F.col("i94_visa")=='Student')\
        .dropDuplicates(["admnum"])\
        .groupby("i94port")\
        .count()\
        .withColumnRenamed('count', 'students_count')
df_res.orderBy("count", ascending=False)\
        .limit(5)\
        .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+
|i94port|students_count|
+-------+--------------+
|    NYC|         46061|
|    CHI|         44082|
|    LOS|         42910|
|    SFR|         25584|
|    BOS|         15165|
+-------+--------------+

In [26]:
df_asia.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1118909

In [29]:
df_asia_merged1 = df_res.join(F.broadcast(df_poe),\
                                df_res.i94port==df_poe.code, 'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_asia_merged1.orderBy("students_count", ascending=False)\
        .limit(5)\
        .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+----+----+-------------+-------------+-------------+
|i94port|students_count|code|port|     locality|     province|    territory|
+-------+--------------+----+----+-------------+-------------+-------------+
|    NYC|         46037| NYC|null|     New York|     New York|United States|
|    CHI|         44088| CHI|null|      Chicago|     Illinois|United States|
|    LOS|         42901| LOS|null|  Los Angeles|   California|United States|
|    SFR|         25584| SFR|null|San Francisco|   California|United States|
|    BOS|         15159| BOS|null|       Boston|Massachusetts|United States|
+-------+--------------+----+----+-------------+-------------+-------------+

In [31]:
df_asia_merged1 = df_asia_merged1.drop("code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_asia_merged2 = df_asia_merged1.join(F.broadcast(df_cities_demo), \
                                    df_asia_merged1.i94port==df_cities_demo.code, \
                                    'inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
df_asia_merged2.orderBy("students_count", ascending=False)\
        .filter(F.col("Race")=="Asian")\
        .select("i94port", "students_count", "locality", \
                "province", "territory", "Race", "Count", \
                "Race_percent_by_city", "Race_rank_by_city")\
        .limit(10)\
        .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+-------------+--------------------+-------------+-----+-------+--------------------+-----------------+
|i94port|students_count|     locality|            province|    territory| Race|  Count|Race_percent_by_city|Race_rank_by_city|
+-------+--------------+-------------+--------------------+-------------+-----+-------+--------------------+-----------------+
|    NYC|         46039|     New York|            New York|United States|Asian|1304564|  15.257335763627571|                1|
|    CHI|         44083|      Chicago|            Illinois|United States|Asian| 195084|  7.1707400987151155|                6|
|    LOS|         42912|  Los Angeles|          California|United States|Asian| 512999|    12.9157208547253|                2|
|    SFR|         25585|San Francisco|          California|United States|Asian| 324034|    37.4685482229746|                4|
|    BOS|         15165|       Boston|       Massachusetts|United States|Asian|  69976|  10.452463071479038|   